In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from sbe_vallib import Validation, BinaryScorer, SupervisedSampler

## Dataset

In [4]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

X, y = make_classification(n_samples=1000, n_classes=2,
                               n_features=5, n_informative=3, n_redundant=0,
                               random_state=0)

X = pd.DataFrame(X)
y = pd.Series(y)

X_train, X_test,  y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)


## Model

In [5]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression().fit(X_train, y_train)

## Sampler Scorer

In [31]:
from sbe_vallib import SupervisedSampler, BinaryScorer

sampler = SupervisedSampler(train={'X': X_train, 'y_true': y_train, 'y_pred': model.predict_proba(X_train)},
                        oos = {'X': X_test, 'y_true': y_test, 'y_pred': model.predict_proba(X_test)})

scorer = BinaryScorer()

In [ ]:
def custom_test(model, scorer, sampler, a = 10, **kwargs):
    metrics = []
    for i in range(a):
        sampler.set_seed(i)
        train = sampler.train
        oos = sampler.oos
        model.fit(X=train["X"], y=train["y_true"])
        y_pred = model.predict_proba(oos['X'])
        metrics.append(scorer.score(oos['y_true'], y_pred))
    
    return {
        "semaphore": "grey",
        "result_dict": {'mean_f1': np.mean([i['f1_score'] for i in metrics])},
        "result_dataframes": [pd.DataFrame(metrics)],
        "result_plots": [],
    }


custom_tests = {
    "test_0_0": {"block": "model_stability", "callable": custom_test, "params": {}},
    "test_0_0_1": {
        "block": "model_stability",
        "callable": custom_test,
        "params": {"a": 100},
    },
}


validor = Validation(model, sampler, scorer, custom_tests=custom_tests,
                         pipeline='../sbe_vallib/validation/table/pipelines/Config_31.xlsx')
res = validor.validate()
res.keys()

In [ ]:
res['test_0_0_1']['result_dataframes'][0]